# 가위바위보 분류기를 만들자

## 1. 필요한 라이브러리 불러오기

In [2]:
from PIL import Image
import os, glob

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


## 2. 이미지 리사이징 하기

In [3]:
import os

def resize_images(img_path):
    images=glob.glob(img_path + "/*.jpg")  
    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
    target_size=(28,28)
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    
    print(len(images), img_path.split('/')[3] ," images resized.")

In [4]:
# 각각 100개의 가위바위보 이미지 리사이징
resize_images("./data/rock_scissor_paper/scissor")
resize_images("./data/rock_scissor_paper/rock")
resize_images("./data/rock_scissor_paper/paper")

100 scissor  images resized.
100 rock  images resized.
100 paper  images resized.


## 3. 가위, 바위, 보 데이터를 읽을 수 있는 함수 만들기

In [5]:
import numpy as np

def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = "./data/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 300 입니다.
x_train shape: (300, 28, 28, 3)
y_train shape: (300,)


## 3) 딥러닝 네트워크 설계하기

In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               80100     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3

## 4) 딥러닝 네트워크 학습시키기

In [7]:
x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

results = model.fit(x_train_reshaped, y_train, epochs=20)

Epoch 1/20
10/10 [==============================] - 0s 4ms/step - loss: 1.1263 - accuracy: 0.2933
Epoch 2/20
10/10 [==============================] - 0s 4ms/step - loss: 1.1024 - accuracy: 0.3367
Epoch 3/20
10/10 [==============================] - 0s 4ms/step - loss: 1.0968 - accuracy: 0.3733
Epoch 4/20
10/10 [==============================] - 0s 4ms/step - loss: 1.0888 - accuracy: 0.4833
Epoch 5/20
10/10 [==============================] - 0s 5ms/step - loss: 1.0829 - accuracy: 0.6233
Epoch 6/20
10/10 [==============================] - 0s 5ms/step - loss: 1.0743 - accuracy: 0.5833
Epoch 7/20
10/10 [==============================] - 0s 5ms/step - loss: 1.0587 - accuracy: 0.5267
Epoch 8/20
10/10 [==============================] - 0s 4ms/step - loss: 1.0327 - accuracy: 0.7533
Epoch 9/20
10/10 [==============================] - 0s 4ms/step - loss: 0.9949 - accuracy: 0.6900
Epoch 10/20
10/10 [==============================] - 0s 4ms/step - loss: 0.9327 - accuracy: 0.8233
Epoch 11/20
10/10 [

## 5) 모델 평가

In [8]:
# x_test, y_test를 만들기
image_dir_path = "./data/rock_scissor_paper/test"
# test data도 train data와 마찬가지로 이미지 resizing 
# 각각 100개의 가위바위보 이미지 리사이징
resize_images("./data/rock_scissor_paper/test/scissor")
resize_images("./data/rock_scissor_paper/test/rock")
resize_images("./data/rock_scissor_paper/test/paper")

(x_test, y_test) = load_data(image_dir_path)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

x_test_reshaped = x_test_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.

100 test  images resized.
100 test  images resized.
100 test  images resized.
학습데이터(x_train)의 이미지 개수는 300 입니다.


In [9]:
# 모델 시험
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 3.0906 - accuracy: 0.3300
test_loss: 3.090582847595215 
test_accuracy: 0.33000001311302185


# 첫 번째 시도 피드백
train accuracy는 87% 인데, test accuracy는 46% 입니다.    
훈련 세트에서는 점수가 굉장히 좋은데 테스트 세트에서는 점수가 굉장히 낮습니다.   
훈련세트에만 잘 맞는 모델은 나중에 새로운 샘플에 대한 예측을 만들 때 잘 동작하지 않을 것입니다.   
데이터가 너무 적기 때문에 데이터의 개수를 늘려 다시 모델 평가

## 1) train, test 데이터 불러오기

In [10]:
# 폴더 내에 있는 모든 이미지 불러오기
image_dir_path = "./data/rock_scissor_paper/second_trial"
dir_list = ['0', '1', '2']

for i in dir_list:
    resize_images(image_dir_path + '/' + i + '/rock')
    resize_images(image_dir_path + '/' + i + '/scissor')
    resize_images(image_dir_path + '/' + i + '/paper')
    print(f'{i}번째 폴더 resize 완료')
    print()

100 second_trial  images resized.
100 second_trial  images resized.
100 second_trial  images resized.
0번째 폴더 resize 완료

100 second_trial  images resized.
100 second_trial  images resized.
100 second_trial  images resized.
1번째 폴더 resize 완료

100 second_trial  images resized.
100 second_trial  images resized.
100 second_trial  images resized.
2번째 폴더 resize 완료



In [11]:
image_dir_path = "./data/rock_scissor_paper/second_trial/"
first = True
for dir in dir_list:
    if first == True:
        (x_data, y_data) = load_data(image_dir_path + "/{}".format(dir))
        first = False
    else:
        (x_data_temp, y_data_temp) = load_data(image_dir_path + "/{}".format(dir))
        x_data = np.vstack((x_data, x_data_temp))     
        y_data = np.hstack((y_data, y_data_temp))

print("x_data shape: {}".format(x_data.shape))
print("y_data shape: {}".format(y_data.shape))

학습데이터(x_train)의 이미지 개수는 300 입니다.
학습데이터(x_train)의 이미지 개수는 300 입니다.
학습데이터(x_train)의 이미지 개수는 300 입니다.
x_data shape: (900, 28, 28, 3)
y_data shape: (900,)


## 2) 데이터 나누기

900장의 사진을 8:2 비율로 train 데이터와 test 데이터로 나눕니다.

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, random_state = 42)
x_train_norm = x_train / 255.0
x_test_norm = x_test / 255.0

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(720, 28, 28, 3)
(720,)
(180, 28, 28, 3)
(180,)


In [15]:
x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

results = model.fit(x_train_reshaped, y_train, epochs=20)

Epoch 1/20
23/23 [==============================] - 0s 5ms/step - loss: 1.8943 - accuracy: 0.4333
Epoch 2/20
23/23 [==============================] - 0s 4ms/step - loss: 0.7300 - accuracy: 0.7306
Epoch 3/20
23/23 [==============================] - 0s 4ms/step - loss: 0.4135 - accuracy: 0.8639
Epoch 4/20
23/23 [==============================] - 0s 4ms/step - loss: 0.2416 - accuracy: 0.9389
Epoch 5/20
23/23 [==============================] - 0s 4ms/step - loss: 0.1831 - accuracy: 0.9403
Epoch 6/20
23/23 [==============================] - 0s 5ms/step - loss: 0.1336 - accuracy: 0.9597
Epoch 7/20
23/23 [==============================] - 0s 5ms/step - loss: 0.1176 - accuracy: 0.9625
Epoch 8/20
23/23 [==============================] - 0s 5ms/step - loss: 0.1018 - accuracy: 0.9597
Epoch 9/20
23/23 [==============================] - 0s 5ms/step - loss: 0.0975 - accuracy: 0.9653
Epoch 10/20
23/23 [==============================] - 0s 5ms/step - loss: 0.0799 - accuracy: 0.9694
Epoch 11/20
23/23 [

In [16]:
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.

# 모델 시험
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

6/6 - 0s - loss: 0.0270 - accuracy: 0.9889
test_loss: 0.027009110897779465 
test_accuracy: 0.9888888597488403


## 최종 결과

train 데이터에서 99퍼센트, test 데이터에서 정확도 98퍼센트가 나왔습니다.
데이터의 수를 약간 늘렸을 뿐인데 test 데이터에서의 정확도 역시 크게 상승했습니다.
처음엔 데이터 수를 늘리지 않고 모델을 좀 더 복잡하게 설계해서 해봤으나 오히려 정확도가 떨어졌습니다.
반면에 데이터 수를 늘리고 모델을 단순하게 설계 했을 때 정확도를 높일 수 있었습니다.